In [2]:
import os
import sys
import time
import serial
import math
import numpy as np
from tqdm import tqdm
from statistics import mean
import pandas as pd
from scipy.interpolate import interp1d
from local_occupancy import sensor_measurements, sensor_distances
from local_occupancy import thymio_coords, sensor_pos_from_center, sensor_angles
%matplotlib inline

sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio

test_functions = False

th = Thymio.serial(port="COM8", refreshing_rate=0.1)

time.sleep(3) # To make sure the Thymio has had time to connect

variables = th.variable_description()

for var in variables : 
    print(var)

{'name': '_id', 'offset': 0, 'size': 1}
{'name': 'event.source', 'offset': 1, 'size': 1}
{'name': 'event.args', 'offset': 2, 'size': 32}
{'name': '_fwversion', 'offset': 34, 'size': 2}
{'name': '_productId', 'offset': 36, 'size': 1}
{'name': 'buttons._raw', 'offset': 37, 'size': 5}
{'name': 'button.backward', 'offset': 42, 'size': 1}
{'name': 'button.left', 'offset': 43, 'size': 1}
{'name': 'button.center', 'offset': 44, 'size': 1}
{'name': 'button.forward', 'offset': 45, 'size': 1}
{'name': 'button.right', 'offset': 46, 'size': 1}
{'name': 'buttons._mean', 'offset': 47, 'size': 5}
{'name': 'buttons._noise', 'offset': 52, 'size': 5}
{'name': 'prox.horizontal', 'offset': 57, 'size': 7}
{'name': 'prox.comm.rx._payloads', 'offset': 64, 'size': 7}
{'name': 'prox.comm.rx._intensities', 'offset': 71, 'size': 7}
{'name': 'prox.comm.rx', 'offset': 78, 'size': 1}
{'name': 'prox.comm.tx', 'offset': 79, 'size': 1}
{'name': 'prox.ground.ambiant', 'offset': 80, 'size': 2}
{'name': 'prox.ground.refl

# adding timer for sampling the proximity sensors datas

In [ ]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False


In [ ]:
from time import sleep
Ts=1
value_proximity=0


def print_sensor_values():
    global value_proximity
    value_proximity=th['prox.horizontal']
    #print("proximity: ",value_proximity)
    #print("accelaration: ",value_acceleration)
    #print("motors' speed: ",value_speed)


#print_sensor_values('prox.horizontal')

In [ ]:
rt = RepeatedTimer(Ts, print_sensor_values)
print(value_proximity)

# A PI controler based on angle error
- (x_a,y_a) = actual position 
- (x_goal,y_goal) = goal position
- angle_actual = actual angle to axis x
then the angle we should correct is angle_goal 

I. Get (x_a, y_a), (x_goal, y_goal), actual_angle

In [18]:
#### à corriger
actual_position = [0,0]
goal = [1,0.3333]
actual_angle = 0 
error_sum = 0


In [19]:
def compute_angle_goal (actual_position,goal,actual_angle):
    return math.atan((goal[1]-actual_position[1])/(goal[0]-actual_position[0]))-actual_angle
    

In [20]:
angle_goal =compute_angle_goal(actual_position,goal,actual_angle)
print(angle_goal)

0.3217205540966482


In [21]:
def follow_the_way_to_dream(actual_position,goal,actual_angle):
    """
    base_speed, kp,ki à tunner
    """
    global error_sum 
    base_speed = 50
    kp = 10
    ki = 10
    error = compute_angle_goal(actual_position,goal,actual_angle)
    error_sum = error_sum + error
    speed_l= base_speed - (kp*error+ki*error_sum)
    speed_r= base_speed + (kp*error+ki*error_sum)
    print("error: ",error)    
    print("speed left: ",speed_l)    
    print("speed right: ",speed_r) 
    
    #call move then 
    
    
    

    """
    if detect_mechant : 
    fait le lacal avoidance
        
    """

In [29]:
follow_the_way_to_dream(actual_position,goal,actual_angle)

error:  0.3217205540966482
speed left:  21.045150131301664
speed right:  78.95484986869833
